# Reservoir Rules

This notebook describes how reservoir release rules are determined for CWatM, and compares observed and simulated reservoir data at several reservoirs in the Nira river sub-basin of the Upper Bhima basin.

Currently, the following data are being used:
 - Reservoir levels
 - Discharge
 - Inflow
 - Irrigation use
 
The observed data for which we compare CWatM simulated reservoir levels are kindly offered on behalf of the National Hydrological Project, India.

Available information related to reservoir releases 

In [1]:
from netCDF4 import Dataset, num2date
import plotly.graph_objects as go
import numpy as np
import datetime
from PIL import Image
import os

## File locations

In [2]:
fuse_folder_local = 'C:/GitHub/FUSE'
fuse_folder_github = 'C:/GitHub/FUSE'
cwatm_folder_local = fuse_folder_local + '/CWATM'
output_folder =  'C:/CWatM_output'

photo_folder = fuse_folder_github + '/Images'
measuredData_folder = fuse_folder_local + '/Data_forNotebooks/Reservoir_Historical/Reservoir level_inflow_floodcontrol'

#Dam_names = ['Vir','Gunjvane', 'NiraDeoghar', 'Bhatghar']
#Inds =  [(164,111), (143,55), (168,66), (159,84)]

#Dam_names = ['Vir', 'Bhatghar']
#Inds =  [(164,111), (159,84)]

Dam_names = ['Vir', 'NiraDeoghar', 'Bhatghar']
Inds =  [(164,111), (168,66), (159,84)]

Reservoirs_Sarati = Dam_names

Vars = [['lakeResStorage', 'Lake Level', 'Reservoir Volume', 'Volume (MCM)', 1000000.],
        ['lakeResOutflowDis', 'Spilling', 'Reservoir Outflow', 'Outflow (m3/s)', 1.], 
        ['lakeResInflowDis', 'positive', 'Reservoir Inflow', 'Inflow (m3/s)', 1.],
        ['act_bigLakeResAbst_alloc', 'Irrigation Use', 'Irrigation Use', 'Volume (MCM)', 1000000.]]

        #['lakeResStorage_alloc', '', 'Total water in segment', 'Volume (MCM)', 1000000.]] 

SIMULATED_nc = []

for var in Vars:
    reservoir_nc_filename = output_folder +'/'+ var[0] + '_daily.nc'
    SIMULATED_nc.append(Dataset(reservoir_nc_filename, 'r'))

FileNotFoundError: [Errno 2] No such file or directory: b'C:/CWatM_output/act_bigLakeResAbst_alloc_daily.nc'

## Introduction

Here, we present the locations and names of the reservoirs of interest.

Ujjani reservoir is only shown for spatial context.

In [ ]:
img = Image.open(photo_folder + '/reservoirs_onSarati.png')
img

In [ ]:
Dates_simulation = num2date(SIMULATED_nc[0].variables['time'][:], units=SIMULATED_nc[0].variables['time'].units)

The below block of code can be activated to list the locations of the reservoir outlet points .

In [ ]:
Storage = SIMULATED_nc[0].variables['lakeResStorage'][1,:,:]
reservoirs = []

for i in range(SIMULATED_nc[0].variables['lat'].shape[0]):
    for j in range (SIMULATED_nc[0].variables['lon'].shape[0]):
        if Storage[i,j] > 0:
            
            reservoirs.append([Storage[i,j], i, j])
            
print(reservoirs)

In [ ]:
DAMS = []

for i in range(len(Vars)): 
    Dams = []
    for inds in Inds:
        Dams.append(SIMULATED_nc[i].variables[Vars[i][0]][:,inds[0], inds[1]]/Vars[i][4])
    DAMS.append(Dams)
    

## CWatM Simulations: 
 - Volumes
 - Discharge
 - Inflow

In [ ]:
for i in range(len(Vars)):
    
    fig = go.Figure()
    Dams = DAMS[i]
    
    for dam_i in range(len(Dams)):
        
        fig.add_trace(go.Scatter(y=Dams[dam_i],
                                 x=Dates_simulation,
                        mode='lines',
                        name=Dam_names[dam_i]))


    fig.update_layout(title = Vars[i][2] +', Simulated',
                           xaxis_title='Days',
                           yaxis_title= Vars[i][3])

    fig.show()

# Analysing observed reservoir data

## Missing files

In [ ]:


import xlrd
import os   

VARS_DATES = []
VARS_LEVELS = []
VARS_Reservoirs_withLevel = []

Reservoirs = os.listdir(measuredData_folder)

for var in Vars:
    
    DATES = []
    LEVELS = []
    Reservoirs_withLevel = []

    for reservoir in Reservoirs:

        find_file = [var[1] +'.xlsx' in i for i in os.listdir(measuredData_folder +'/'+ reservoir)]

        if True in find_file:

            filename = os.listdir(measuredData_folder +'/'+ reservoir)[find_file.index(True)]

            book = xlrd.open_workbook(measuredData_folder  +'/'+ reservoir +'/'+ filename)
            sheet = book.sheet_by_index(0)
            num_rows = sheet.nrows

            Dates_fromExcel = [xlrd.xldate_as_tuple(int(sheet.cell(row,0).value), 0) for row in range(2, num_rows)]
            Dates = [datetime.datetime(d[0], d[1], d[2]) for d in Dates_fromExcel]

            Levels = [sheet.cell(row, 1).value for row in range(2, num_rows)]


            DATES.append(Dates)
            LEVELS.append(Levels)
            Reservoirs_withLevel.append(reservoir)

        else:
            print('Missing file: '+ var[2] +': '+ reservoir)
                
    VARS_DATES.append(DATES)
    VARS_LEVELS.append(LEVELS)
    VARS_Reservoirs_withLevel.append(Reservoirs_withLevel)


In [ ]:
def level_to_volume(level, reservoir):

    if level == '':
        volume = ''
    else:
        if reservoir == 'Veer' or reservoir =='Vir':
            volume = 0.5312*level**2 - 591.26*level + 164526
        elif reservoir == 'Bhatghar':
            
            # -0,0003x4 + 0,7976x3 - 719,23x2 + 287999x - 4E+07
            # Jan 25 commented out volume = 0.5129241275659454*level**2 -602.0396530087479*level + 176662.66875475977
            #volume = 0.4836*level**2 -566.54*level + 165930
            volume = 0.4836*level**2 -566.54*level + 165923
            
            
            # volume = 0.4641*level**2 - 542.84*level + 158709
        elif reservoir in ['NiraDeoghar', 'NiraDevdhar']:
            volume = 0.1543*level**2 - 191.37*level + 59331
        elif reservoir == 'Gunjvane':
            volume = y = 0.0988*level**2 - 137.62*level + 47935

    
    return volume


## Missing values

In [ ]:
VARS_MISSING_dates = []
VARS_LEVELS_removeNoData = []
VARS_DATES_removeNoData = []


for var_i in range(len(Vars)):
    
    MISSING_dates = []
    LEVELS_removeNoData = []
    DATES_removeNoData = []

    for res_i in range(len(VARS_Reservoirs_withLevel[var_i])):
        
        missing_dates = []
        Levels_removeNoData = []
        Dates_removeNoData = []

        Levels = VARS_LEVELS[var_i][res_i]
        Dates = VARS_DATES[var_i][res_i]


        for i in range(len(Levels)):

            if Levels[i] == '' or Levels[i] == 0:
                missing_dates.append(Dates[i])
            else:
                Levels_removeNoData.append(Levels[i])
                Dates_removeNoData.append(Dates[i])

        percent_missing = int(len(missing_dates)/len(Levels)*100)

        print(Vars[var_i][2])
        print(VARS_Reservoirs_withLevel[var_i][res_i])
        #print(Reservoirs_withLevel[res_i])
        
        #print(Vars[var_i][2] + ' for '+ Reservoirs_withLevel[res_i] +': '+str(percent_missing) + ' % of the values are missing.')
        print(Vars[var_i][2] + ' for '+ VARS_Reservoirs_withLevel[var_i][res_i] +': '+str(percent_missing) + ' % of the values are missing.')

        
        MISSING_dates.append(missing_dates)
        LEVELS_removeNoData.append(Levels_removeNoData)
        DATES_removeNoData.append(Dates_removeNoData)                    
            
    VARS_MISSING_dates.append(MISSING_dates)
    VARS_LEVELS_removeNoData.append(LEVELS_removeNoData)
    VARS_DATES_removeNoData.append(DATES_removeNoData)
    print('\n')
                            


In [ ]:
VARS_VOLUMES_removeNoData = []

for var_i in range(len(Vars)):
    
    VOLUMES_removeNoData = []
    
    Reservoirs_withLevel = VARS_Reservoirs_withLevel[var_i]
    
    for res_i in range(len(Reservoirs_withLevel)):
        if Reservoirs_withLevel[res_i] in Reservoirs_Sarati:
            
            Levels = VARS_LEVELS_removeNoData[var_i][res_i]
            
            if Vars[var_i][1] == 'Lake Level':
                Volumes = [level_to_volume(level, Reservoirs_withLevel[res_i]) for level in Levels]
                
            elif Vars[var_i][1] == 'Irrigation Use':
                Volumes = np.array(Levels)*60*60*24/1000000
            else:
                Volumes = Levels
                
            Volumes_corrected = [volume if volume=='' or volume<3500 else '' for volume in Volumes]
                
            VOLUMES_removeNoData.append(Volumes_corrected)
            
            #print(Vars[var_i][2] + ' for '+ Reservoirs_withLevel[res_i] + ': The maximum is ' + str(max(Volumes_corrected)))
            #print(Vars[var_i][2] + ' for '+ Reservoirs_withLevel[res_i] + ': The minimum is ' + str(min(Volumes_corrected)))

        else:
            VOLUMES_removeNoData.append([])
    
    VARS_VOLUMES_removeNoData.append(VOLUMES_removeNoData)
        

# Visualisations
## Reservoir Volumes, Outflows, and Inflows

In [ ]:
VARS_VOLUMES = []

for var_i in range(len(Vars)):
    
    VOLUMES = []
    
    Reservoirs_withLevel = VARS_Reservoirs_withLevel[var_i]
    
    for res_i in range(len(Reservoirs_withLevel)):
        if Reservoirs_withLevel[res_i] in Reservoirs_Sarati:
            
            Levels = VARS_LEVELS[var_i][res_i]
            
            if Vars[var_i][1] == 'Lake Level':
                Volumes = [level_to_volume(level, Reservoirs_withLevel[res_i]) for level in Levels]
                
            elif Vars[var_i][1] == 'Irrigation Use':
                Volumes = np.array(Levels)*60*60*24/1000000
            else:
                Volumes = Levels
                
            Volumes_corrected = [volume if volume=='' or volume<3500 else '' for volume in Volumes]
                
            VOLUMES.append(Volumes_corrected)
            
            #print(Vars[var_i][2] + ' for '+ Reservoirs_withLevel[res_i] + ': The maximum is ' + str(max(Volumes_corrected)))
            #print(Vars[var_i][2] + ' for '+ Reservoirs_withLevel[res_i] + ': The minimum is ' + str(min(Volumes_corrected)))

        else:
            VOLUMES.append([])
    
    VARS_VOLUMES.append(VOLUMES)
        

# Visualisations
## Reservoir Volumes, Outflows, and Inflows

In [ ]:
for var_i in range(len(Vars)):
    
    Reservoirs_withLevel = VARS_Reservoirs_withLevel[var_i]
    
    for res_i in range(len(VARS_Reservoirs_withLevel[var_i])):
        
        if Reservoirs_withLevel[res_i] in Reservoirs_Sarati: 
            
            Dates = VARS_DATES[var_i][res_i]
            Volumes = VARS_VOLUMES[var_i][res_i]
            fig = go.Figure()

            fig.add_trace(go.Scatter(x = VARS_DATES[var_i][res_i],
                                     y = VARS_VOLUMES[var_i][res_i], 
                                     mode='lines',
                                     name='Measured'))

            fig.add_trace(go.Scatter(x = Dates_simulation, #x,
                                     y = DAMS[var_i][Dam_names.index(Reservoirs_withLevel[res_i])],
                                     mode='lines',
                                     name='Simulated'))

            fig.update_layout(title= Vars[var_i][2] +': '+ Reservoirs_withLevel[res_i],
                                   xaxis_title='Date (Day)',
                                   yaxis_title= Vars[var_i][3])

            fig.show()

In [ ]:
Reservoirs_withLevel[16]

In [ ]:
#for var_i in range(len(Vars)):
for var_i in [3]:
    
    Reservoirs_withLevel = VARS_Reservoirs_withLevel[var_i]
    
    for res_i in range(len(VARS_Reservoirs_withLevel[var_i])):
        
        if Reservoirs_withLevel[res_i] in Reservoirs_Sarati: 
            
            Dates = VARS_DATES[var_i][res_i]
            Volumes = VARS_VOLUMES[var_i][res_i]
            fig = go.Figure()


            fig.add_trace(go.Scatter(x = VARS_DATES[var_i][res_i],
                                     y = VARS_VOLUMES[var_i][res_i], 
                                     mode='lines',
                                     name='Measured'))

            fig.add_trace(go.Scatter(x = Dates_simulation, #x,
                                     y = DAMS[var_i][Dam_names.index(Reservoirs_withLevel[res_i])],
                                     mode='lines',
                                     name='Simulated'))

            fig.update_layout(title= Vars[var_i][2] +': '+ Reservoirs_withLevel[res_i],
                                   xaxis_title='Date (Day)',
                                   yaxis_title= Vars[var_i][3])

            fig.show()


In [ ]:
year_first = Dates[0].year
year_last = Dates[-1].year
years_i = []
            
for year in range(year_first, year_last):
    years_i.append(Dates.index(datetime.datetime(year, 6, 1, 0, 0)))

# datetime.datetime(1964, 6, 1, 0, 0)
# datetime.datetime(2008, 5, 31, 0, 0)

In [ ]:
Dates[0]

In [ ]:
from statistics import mean

fig = go.Figure()
for i in range(len(years_i)-1):
    fig.add_trace(go.Scatter(x = Dates[years_i[0]: years_i[1]],
                             y = Volumes[years_i[i]: years_i[i+1]],
                             mode='lines',
                             name='Measured'))

Dates[years_i[0]: years_i[1]]

Daily_discharge_average = [mean([Volumes[years_i[year_i] : years_i[year_i + 1]][day] for year_i in range(len(years_i)-1)]) for day in range(365)]

fig.add_trace(go.Scatter(x = Dates[years_i[0]: years_i[1]],
                         y = Daily_discharge_average,
                         mode='lines',
                         name='Measured'))

fig.show()

fig2 = go.Figure()
for i in range(1): #(len(years_i)-1):
    fig2.add_trace(go.Scatter(x = Dates[years_i[0]: years_i[1]],
                             y = Volumes[years_i[i]: years_i[i+1]],
                             mode='lines',
                             name='Measured'))

Daily_discharge_average = [mean([Volumes[years_i[year_i] : years_i[year_i + 1]][day] for year_i in range(len(years_i)-1)]) for day in range(365)]

fig2.add_trace(go.Scatter(x = Dates[years_i[0]: years_i[1]],
                         y = Daily_discharge_average,
                         mode='lines',
                         name='Measured'))

fig2.show()

In [ ]:
# Irrigation discharge normalized by reservoir volume

#Volume_np = np.array(VARS_VOLUMES[0][18]) #Volume
Volumes_woBlank = []
#Irrigation_np = VARS_VOLUMES[3][16] #Irrigation
Irrigation_woBlank = []
#Irr_overVolume = Irrigation_np/Volume_np


for i in VARS_VOLUMES[0][18]:
    if i=='':
        Volumes_woBlank.append(0)
    else:
        Volumes_woBlank.append(i)
        
for i in VARS_VOLUMES[3][16]:
    if i=='':
        Irrigation_woBlank.append(0)
    else:
        Irrigation_woBlank.append(i)
        
Volume_np = np.array(Volumes_woBlank)
Irrigation_np = np.array(Irrigation_woBlank)
 
print(len(Volume_np))
print(len(Irrigation_np))

#x = Irrigation_np/Volume_np
Irr_overVolume = [Irrigation_np[i]/Volume_np[i] if Volume_np[i]>0 else 0 for i in range(len(Volume_np))]



In [ ]:
from statistics import mean, median

fig = go.Figure()

for i in range(len(years_i)-1):
    fig.add_trace(go.Scatter(x = Dates[years_i[0]: years_i[1]],
                         y = Irr_overVolume[years_i[i]: years_i[i+1]],
                         mode='lines',
                         name='Measured'))



Dates[years_i[0]: years_i[1]]

Daily_discharge_average = [mean([Irr_overVolume[years_i[year_i] : years_i[year_i + 1]][day] for year_i in range(len(years_i)-1)]) for day in range(365)]

fig.add_trace(go.Scatter(x = Dates[years_i[0]: years_i[1]],
                         y = Daily_discharge_average,
                         mode='lines',
                         name='Mean'))

Daily_discharge_median = [median([Irr_overVolume[years_i[year_i] : years_i[year_i + 1]][day] for year_i in range(len(years_i)-1)]) for day in range(365)]

fig.add_trace(go.Scatter(x = Dates[years_i[0]: years_i[1]],
                         y = Daily_discharge_median,
                         mode='lines',
                         name='Median'))

#fig2 = go.Figure(data=[go.Box([Volumes[years_i[year_i] : years_i[year_i + 1]][day] for year_i in range(len(years_i)-1)]) for day in range(365)])
#fig2 = go.Figure(data=[go.Box([Volumes[years_i[0] : years_i[1]][day], Volumes[years_i[1] : years_i[2]][day]]) for day in range(2)])

#fig2.show()

In [ ]:
DAYS_of_year = [i.timetuple().tm_yday for i in Dates[years_i[0]: years_i[1]]]

In [ ]:
tuples = [[DAYS_of_year[i], Daily_discharge_median[i]] for i in range(len(DAYS_of_year))]

In [ ]:
tuples.sort()

In [ ]:
fractions = [i[1] for i in tuples]

In [ ]:
len(fractions)

In [ ]:
import xlsxwriter

# Create a workbook and add a worksheet.
workbook = xlsxwriter.Workbook('Test_irrigation.xlsx')
worksheet = workbook.add_worksheet('Veer_median')

# Some data we want to write to the worksheet.
x = Dates[years_i[0]: years_i[1]]
y = Daily_discharge_median

x_forExcel = ["{0:0=2d}".format(d.day)+'/'+"{0:0=2d}".format(d.month) for d in x]

# Start from the first cell. Rows and columns are zero indexed.
row = 0
col = 0

# Iterate over the data and write it out row by row.
for i in range(len(x)):
    worksheet.write(row, col,     str(x_forExcel[i]))
    worksheet.write(row, col + 1, str(y[i]))
    row += 1

worksheet = workbook.add_worksheet('Veer_mean')

# Some data we want to write to the worksheet.
#x = Dates[years_i[0]: years_i[1]]
y = Daily_discharge_average



# Start from the first cell. Rows and columns are zero indexed.
row = 0
col = 0

# Iterate over the data and write it out row by row.
for i in range(len(x)):
    worksheet.write(row, col,     str(x_forExcel[i]))
    worksheet.write(row, col + 1, str(y[i]))
    row += 1

workbook.close()

##TODO: Leap years

In [ ]:


filename = 'Test_irrigation.xlsx'

book = xlrd.open_workbook(filename)
sheet = book.sheet_by_index(0)
num_rows = sheet.nrows

Dates_fromExcel = [sheet.cell(row,0).value for row in range(0, num_rows)]
#Dates_fromExcel2 = [i.split()[0].split('-') for i in Dates_fromExcel]
#Dates = [datetime.datetime(int(d[0]), int(d[1]), int(d[2])) for d in Dates_fromExcel2]

#Levels = [sheet.cell(row, 1).value for row in range(2, num_rows)]

In [ ]:
#Ratios_fromExcel = [float(sheet.cell(row, 1).value) for row in range(0, num_rows)]
Dates_fromExcel

In [ ]:
from statistics import mean

fig = go.Figure()

for i in range(len(years_i)-1):

    fig.add_trace(go.Scatter(x = [0,1],
                             y = [mean(Volumes[years_i[i]: years_i[i+1]])]*2,
                             mode='lines',
                             name='Measured'))

Daily_discharge_average = [mean([Volumes[years_i[year_i] : years_i[year_i + 1]][day] for year_i in range(len(years_i)-1)]) for day in range(365)]

fig.add_trace(go.Scatter(x = [0,1],
                         y = [mean(Daily_discharge_average)]*2,
                         mode='lines',
                         name='Measured'))

fig.add_trace(go.Box(y = [mean(Volumes[years_i[i]: years_i[i+1]]) for i in range(len(years_i)-1)]))

fig.show()



fig2 = go.Figure()

for i in range(len(years_i)-1):

    fig2.add_trace(go.Scatter(x = [0,1],
                             y = [mean(Irr_overVolume[years_i[i]: years_i[i+1]])]*2,
                             mode='lines',
                             name='Measured'))

Daily_discharge_average = [mean([Irr_overVolume[years_i[year_i] : years_i[year_i + 1]][day] for year_i in range(len(years_i)-1)]) for day in range(365)]

fig2.add_trace(go.Scatter(x = [0,1],
                         y = [mean(Daily_discharge_average)]*2,
                         mode='lines',
                         name='Measured'))

fig2.add_trace(go.Scatter(x = [0,1],
                         y = [mean(Daily_discharge_median)]*2,
                         mode='lines',
                         name='Median'))

fig2.add_trace(go.Box(y = [mean(Irr_overVolume[years_i[i]: years_i[i+1]]) for i in range(len(years_i)-1)]))

fig2.show()




In [ ]:
import plotly.express as px
df = px.data.tips()
fig = px.box(y=[mean(Volumes[years_i[i]: years_i[i+1]]) for i in range(len(years_i)-1)])
fig.show()

In [ ]:
y = [[Volumes[years_i[year_i] : years_i[year_i + 1]][day] for year_i in range(len(years_i)-1)] for day in range(2)]
print(y)

In [ ]:
fig = go.Figure()

for day in range(365):

    fig.add_trace(go.Box(y = [Volumes[years_i[year_i] : years_i[year_i + 1]][day] for year_i in range(len(years_i)-1)])) 


fig.show()

fig2 = go.Figure()

for day in range(365):

    fig2.add_trace(go.Box(y = [Irr_overVolume[years_i[year_i] : years_i[year_i + 1]][day] for year_i in range(len(years_i)-1)])) 


fig2.show()

In [ ]:
import operator

import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures


x = np.array(range(365)) #Dates[years_i[0]: years_i[1]]
y = np.array(Daily_discharge_median)

# transforming the data to include another axis
x = x[:, np.newaxis]
y = y[:, np.newaxis]

polynomial_features= PolynomialFeatures(degree=8)
x_poly = polynomial_features.fit_transform(x)

model = LinearRegression()
model.fit(x_poly, y)
y_poly_pred = model.predict(x_poly)


rmse = np.sqrt(mean_squared_error(y,y_poly_pred))
r2 = r2_score(y,y_poly_pred)
print(rmse)
print(r2)

plt.scatter(x, y, s=10)
# sort the values of x before line plot
sort_axis = operator.itemgetter(0)
sorted_zip = sorted(zip(x,y_poly_pred), key=sort_axis)
x, y_poly_pred = zip(*sorted_zip)
plt.plot(x, y_poly_pred, color='m')
plt.show()

In [ ]:
import operator

import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures


x = np.array(range(365))
y = np.array(Daily_discharge_average)

# transforming the data to include another axis
x = x[:, np.newaxis]
y = y[:, np.newaxis]

polynomial_features= PolynomialFeatures(degree=7)
x_poly = polynomial_features.fit_transform(x)

model = LinearRegression()
model.fit(x_poly, y)
y_poly_pred = model.predict(x_poly)

rmse = np.sqrt(mean_squared_error(y,y_poly_pred))
r2 = r2_score(y,y_poly_pred)
print(rmse)
print(r2)

plt.scatter(x, y, s=10)
# sort the values of x before line plot
sort_axis = operator.itemgetter(0)
sorted_zip = sorted(zip(x,y_poly_pred), key=sort_axis)
x, y_poly_pred = zip(*sorted_zip)
plt.plot(x, y_poly_pred, color='m')
plt.show()